In [8]:
# 1. 首先导入必要的库
from tree_sitter import Language, Parser

from RRD.railroad import Diagram, Terminal, NonTerminal, Choice, Sequence, Optional, OneOrMore, ZeroOrMore

# 2. 设置EBNF解析器
# 假设您的EBNF解析器已经编译并位于正确的位置
# Language.build_library(
#     'build/my-ebnf3.so',
#     ['./tree-sitter-ebnf']
# )
EBNF_LANGUAGE = Language('./build/my-ebnf3.so', 'EBNF')
parser = Parser()
parser.set_language(EBNF_LANGUAGE)

# 3. 读取EBNF文件
with open('tree-sitter-ebnf-generator/examples/lua/lua.ebnf', 'r') as f:
    ebnf_text = f.read()

# 4. 解析EBNF
tree = parser.parse(bytes(ebnf_text, 'utf8'))

# 5. 定义visit函数来处理语法树
def visit(node, diagrams = {}):
    # 添加调试信息
    print(f"Processing node type: {node.type}")
    
    match(node.type):
        case 'source_file':
            for i in range(len(node.children)):
                diagrams = visit(node.children[i], diagrams)
            return diagrams
        case 'rule':
            name = node.children[0].text.decode()
            diagrams[name] = Diagram(visit(node.children[2])).setPhenomenon('rule')
            return diagrams
        case 'rhs':
            return visit(node.children[0]).setPhenomenon('rhs')
        case 'sequence':
            blocks = []
            for child in node.children:
                blocks.append(visit(child))
            return Sequence(*blocks).setPhenomenon('sequence')
        case 'statements':
            return visit(node.children[0]).setPhenomenon('statements')
        case 'parenthesizedStatement':
            return visit(node.children[1]).setPhenomenon('parenthesized')
        case 'quantifiedStatement':
            return visit(node.children[0]).setPhenomenon('quantified')
        case 'selectionStatement':
            blocks = []
            for child in node.children:
                if child.type == 'sequenceNoSelection':
                    blocks.append(visit(child))
            return Choice(len(blocks)//2, *blocks).setPhenomenon('selection')
        case 'sequenceNoSelection':
            blocks = []
            for child in node.children:
                blocks.append(visit(child))
            return Sequence(*blocks).setPhenomenon('sequence_no_selection')
        case 'statementsNoSelection':
            return visit(node.children[0]).setPhenomenon('statements_no_selection')
        case 'optional':
            return Optional(visit(node.children[0])).setPhenomenon('optional')
        case 'oneOrMore':
            return OneOrMore(visit(node.children[0])).setPhenomenon('one_or_more')
        case 'zeroOrMore':
            return ZeroOrMore(visit(node.children[0])).setPhenomenon('zero_or_more')
        case 'quantifierBase':
            return visit(node.children[0]).setPhenomenon('quantifier_base')
        case 'identifier':
            return Terminal(node.text.decode()).setPhenomenon('identifier')
        case _:
            if len(node.children) > 0:
                print(f'Not implemented node type: {node.type}')
                print(f'Children types: {[child.type for child in node.children]}')
                # 返回第一个子节点的结果，而不是直接返回None
                return visit(node.children[0])
            else:
                # 如果是叶子节点，返回一个默认的Terminal
                return Terminal(node.text.decode()).setPhenomenon('unknown')

# 6. 处理语法树并生成图表
diagrams = visit(tree.root_node)

# 7. 生成SVG输出
# 假设我们要生成第一个规则的图表
first_rule = list(diagrams.values())[0]

# 8. 将SVG保存到文件
with open('output.svg', 'w') as f:
    f.write('''<?xml version="1.0" encoding="UTF-8"?>
    <!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN" "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
    ''')
    first_rule.writeStandalone(f.write)

# 9. 在Jupyter中显示SVG
from IPython.display import SVG
SVG('output.svg')

Not implemented: ERROR


AttributeError: 'NoneType' object has no attribute 'values'